# Reverse Polish Notation

In [2]:
from typing import Callable

## Numbers

In [3]:
unary: set = {"--"}
binary: set = {"+", "-", "*", "/"}


def simple_rpn_to_infix(expression: list):
    global unary, binary
    stack = []

    for token in expression:
        if token.isdigit():
            # Push number onto the stack
            stack.append(token)
            continue
        if token in binary:
            operand2 = stack.pop()
            operand1 = stack.pop()
            stack.append(f"({operand1} {token} {operand2})")
            continue
        if token in unary:
            operand = stack.pop()
            stack.append(f"({token}{operand})")

    return stack[0]


rpn_expression = "3 4 + 2 * 7 -- /"
infix_expression = simple_rpn_to_infix(rpn_expression.split())
print("Infix Expression:", infix_expression)

Infix Expression: (((3 + 4) * 2) / (--7))


## RegEx

In [ ]:
def unary_lambda(symbol: str) -> Callable[[str], str]:
    return lambda a: f"({a}){symbol}"


letters = "abcdefghijklmnopqrstuvwxyz"
digits = "0123456789"
symbols = "%#@~'\"<>/,&:;№!{}"

quantifiers: set[str] = {"*", "+", "?"}

unary_regex: dict[str, Callable[[str], str]] = {
    **{symbol: unary_lambda(symbol) for symbol in quantifiers},
    **{
        f"{symbol}?": unary_lambda(f"{symbol}?") for symbol in quantifiers
    },  # greedy versions
}
binary_regex: dict[str, Callable[[str, str], str]] = {
    "concat": lambda a, b: a + b,  # concatenation
    "|": lambda a, b: f"{a}|{b}",
}
many_to_one_regex: dict[str, Callable[[list[str]], str]] = {
    "[]": lambda a: "[" + "".join([f"({x})" for x in a]) + "]",  # set
    "^[]": lambda a: "[^" + "".join([f"({x})" for x in a]) + "]",  # not in set
    "concat_all": lambda a: "".join(a),
}

metacharacters_regex: set[str] = {
    ".",
    "^",
    "$",
}

special_sequences: set[str] = {r"\d", r"\s", r"\w"}

operands_regex: set[str] = {
    *metacharacters_regex,
    *[rf"\{s}" for s in metacharacters_regex],
    *letters,
    *letters.upper(),
    *symbols,
    *digits,
    *[rf"\{s}" for s in quantifiers],
    *special_sequences,
    r"\\",
}


def rpn_to_infix_regex(expression: list):
    global unary_regex, binary_regex, operands_regex
    stack = []

    for token in expression:
        print(stack)
        if token in many_to_one_regex:
            stack = [many_to_one_regex[token](list(reversed(stack)))]
            continue

        if token in binary_regex:
            operand2 = stack.pop()
            operand1 = stack.pop()
            stack.append(binary_regex[token](operand1, operand2))
            continue
        if token in unary_regex:
            operand = stack.pop()
            stack.append(unary_regex[token](operand))
            continue
        if token in operands_regex:
            stack.append(token)
            continue
        raise RuntimeError(f"Operand '{token}' is unknown")
    return stack[0]


rpn_expression_regex = ["a", "b", "c", "concat", "[]"]
rpn_expression_regex = ["a", "b", "c", "concat_all"]
infix_expression_regex = rpn_to_infix_regex(rpn_expression_regex)
print("Infix Expression:", infix_expression_regex)

Infix Expression: cba
